In [ ]:
# To convert from NFA to DFA using subset construction method
# 1. Epsilon closure of the initial state
# 2. For each state in the epsilon closure, find the transition on each input symbol
# 3. Epsilon closure of the states reached in step 2
# 4. Repeat step 2 and 3 until no new states are reached
# 5. The states reached in step 4 are the states of the DFA
# 6. The transition function of the DFA is the transitions found in step 2 and 3

In [78]:
def Split_range(range_string):
    if '-' not in range_string:
        return [range_string]
    start, end = range_string.split('-')
    list_range = []
    try :
        start = int(start)
        end = int(end)
        for i in range(start, end + 1):
            list_range.append(i)
        return list_range
    except ValueError:
        for i in range(ord(start), ord(end) + 1):
            list_range.append(chr(i))
        return list_range

range_string = 'a-z'
Symbols = Split_range(range_string)
print(Symbols)

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [162]:
import json
def build_nfa_from_json(path):
    with open(path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    start_state = []
    start_state.append(data['startingState'])
    accept_states = []
    for state in data:
        if state == 'startingState' :
            continue
        if data[state]['isTerminatingState'] :
            accept_states.append(state)
    Symbols = set()
    nfa = {}
    for state in data:
        if state == 'startingState' :
            continue
        nfa[state] = {}
        for key in data[state] :
            if key == 'isTerminatingState' :
                continue
            nfa[state] = {key : set()}
            for k in data[state][key]:
                nfa[state][key].add(k)
            if key != 'ε' :
                Symbols.add(key)
    
    return nfa,list(Symbols),start_state , accept_states
nfa, Symbols ,start_state , accept_states = build_nfa_from_json('RE_NFA/output.json')
print('NFA : ',nfa)
print('Symbols : ' ,Symbols)
print('Start State : ' , start_state)
print('Accept State : ' , accept_states)   
print(len(nfa))

NFA :  {'S1': {'a': {'S2'}}, 'S2': {'ε': {'S3'}}, 'S3': {'b': {'S4'}}, 'S4': {'ε': {'S11'}}, 'S11': {'ε': {'S12', 'S9'}}, 'S12': {'ε': {'S15'}}, 'S9': {'ε': {'S5', 'S7'}}, 'S10': {'ε': {'S11', 'S12'}}, 'S7': {'c': {'S8'}}, 'S8': {'ε': {'S10'}}, 'S5': {'b': {'S6'}}, 'S6': {'ε': {'S10'}}, 'S15': {'ε': {'S13'}}, 'S16': {}, 'S13': {'d': {'S14'}}, 'S14': {'ε': {'S16', 'S15'}}}
Symbols :  ['c', 'b', 'a', 'd']
Start State :  ['S1']
Accept State :  ['S16']
16


In [88]:
set_of_states = list()
for symbol in Symbols:
    for s in Split_range(symbol):
        set_of_states.append(s)
print(set(set_of_states))

{'d', 'c', 'a', 'e', 'b'}


In [357]:
Symbols = ['a', 'b', 'c']
accept_states = {4}
start_state = {0}

In [163]:
def epsilon_closure(nfa, states):
    #states = {states} if isinstance(states, str) else set(states)
    closure = set(states)
    stack = list(states)
    while stack:
        state = stack.pop()
        if 'ε' in nfa[state]:
            for s in nfa[state]['ε']:
                if s not in closure:
                    closure.add(s)
                    stack.append(s)
    return closure

def move(nfa, states, symbol):
    moves = set()
    for state in states:
        if symbol in nfa[state]:
            moves.update(nfa[state][symbol])
    return moves

# nfa = {
#     0: {'a': {2} , 'ε': {3} , 'b': {4}},
#     1: {'a': {3}, 'b': {2}},
#     2: {'a': {2} , 'b': {4}},
#     3: {'a' : {3} , 'b': {1}},
#     4: {'ε': {3} , 'b': {1} , 'a': {2}},
# }
D_states = epsilon_closure(nfa,start_state) # {0, 1}
print(D_states)
D_transitions = {}
def convert_dfa_from_nfa(nfa , D_states , D_transitions) :
    stack = [D_states]
    while stack:
        D_states = stack.pop()
        for symbol in Symbols:
            moves = move(nfa, D_states, symbol)
            moves = epsilon_closure(nfa, moves)
            if moves:
                if (frozenset(D_states), symbol) not in D_transitions:
                    D_transitions[(frozenset(D_states), symbol)] = moves
                    stack.append(moves)
    return D_transitions


D_transitions = convert_dfa_from_nfa(nfa , D_states , D_transitions)

D_states = set(D_transitions.keys())
set_of_states = set()
dict_of_states = {}
for D_state in D_states:
    set_of_states.add(D_state[0])
for state in (set_of_states):
    for accept_state in accept_states:
        if accept_state in state:
            dict_of_states[str(set(state))] = True
            break
        else:
            dict_of_states[str(set(state))] = False
print(dict_of_states)
print(D_transitions)
#D_transitions = {(frozenset({0, 1, 2}), 'a'): {2, 3}, (frozenset({0, 1, 2}), 'b'): {3, 4}, (frozenset({3, 4}), 'a'): {2}, (frozenset({3, 4}), 'b'): {1, 2}, (frozenset({1, 2}), 'a'): {3}, (frozenset({1, 2}), 'b'): {3, 4}, (frozenset({3}), 'b'): {1, 2}, (frozenset({2}), 'b'): {3, 4}, (frozenset({2, 3}), 'b'): {1, 2, 3, 4}, (frozenset({1, 2, 3, 4}), 'a'): {2, 3}, (frozenset({1, 2, 3, 4}), 'b'): {1, 2, 3, 4}, (frozenset({3}), 'a'): {3, 4}, (frozenset({2}), 'a'): {1, 2, 3, 4}, (frozenset({2, 3}), 'a'): {0, 1, 2}}

{'S1'}
{"{'S15', 'S11', 'S12', 'S5', 'S13', 'S9', 'S4', 'S7'}": False, "{'S8', 'S15', 'S11', 'S10', 'S12', 'S13', 'S5', 'S9', 'S7'}": False, "{'S3', 'S2'}": False, "{'S1'}": False, "{'S16', 'S13', 'S15', 'S14'}": True, "{'S15', 'S6', 'S11', 'S10', 'S12', 'S13', 'S5', 'S9', 'S7'}": False}
{(frozenset({'S1'}), 'a'): {'S3', 'S2'}, (frozenset({'S3', 'S2'}), 'b'): {'S15', 'S11', 'S12', 'S5', 'S13', 'S9', 'S4', 'S7'}, (frozenset({'S15', 'S11', 'S12', 'S5', 'S13', 'S9', 'S4', 'S7'}), 'c'): {'S8', 'S15', 'S11', 'S10', 'S12', 'S13', 'S5', 'S9', 'S7'}, (frozenset({'S15', 'S11', 'S12', 'S5', 'S13', 'S9', 'S4', 'S7'}), 'b'): {'S15', 'S6', 'S11', 'S10', 'S12', 'S13', 'S5', 'S9', 'S7'}, (frozenset({'S15', 'S11', 'S12', 'S5', 'S13', 'S9', 'S4', 'S7'}), 'd'): {'S16', 'S13', 'S15', 'S14'}, (frozenset({'S16', 'S13', 'S15', 'S14'}), 'd'): {'S16', 'S13', 'S15', 'S14'}, (frozenset({'S15', 'S6', 'S11', 'S10', 'S12', 'S13', 'S5', 'S9', 'S7'}), 'c'): {'S8', 'S15', 'S11', 'S10', 'S12', 'S13', 'S5', 'S9', 'S7'}

In [164]:
new_D_transitions = {}

# Iterate through the original dictionary
for key, value in D_transitions.items():
    # Extract state and symbol from the key
    state = ', '.join(str(s) for s in key[0])
    #print(state)
    state = "{'" + "', '".join(state.strip("{}").split(", ")) + "'}"
    #print(state)
    symbol = key[1]
    # Construct new key in the desired format
    new_key = state, symbol
    new_D_transitions[new_key] = value

print(new_D_transitions)
D_transitions = new_D_transitions

real_states = list()
for one in (D_transitions.keys()) :
    if (one[0]) not in real_states:
        elements = one[0].strip("{}").split(", ")
        converted = "{'" + "', '".join(elements) + "'}"
        real_states.append(one[0])
print("Real States: ",real_states)  
print(len(real_states))
new_names_states = {}
for i in range(len(real_states)):
    new_names_states[str(real_states[i])] = f'S{i}'
print(new_names_states)   


{("{'S1'}", 'a'): {'S3', 'S2'}, ("{'S3', 'S2'}", 'b'): {'S15', 'S11', 'S12', 'S5', 'S13', 'S9', 'S4', 'S7'}, ("{'S15', 'S11', 'S12', 'S5', 'S13', 'S9', 'S4', 'S7'}", 'c'): {'S8', 'S15', 'S11', 'S10', 'S12', 'S13', 'S5', 'S9', 'S7'}, ("{'S15', 'S11', 'S12', 'S5', 'S13', 'S9', 'S4', 'S7'}", 'b'): {'S15', 'S6', 'S11', 'S10', 'S12', 'S13', 'S5', 'S9', 'S7'}, ("{'S15', 'S11', 'S12', 'S5', 'S13', 'S9', 'S4', 'S7'}", 'd'): {'S16', 'S13', 'S15', 'S14'}, ("{'S16', 'S13', 'S15', 'S14'}", 'd'): {'S16', 'S13', 'S15', 'S14'}, ("{'S15', 'S6', 'S11', 'S10', 'S12', 'S13', 'S5', 'S9', 'S7'}", 'c'): {'S8', 'S15', 'S11', 'S10', 'S12', 'S13', 'S5', 'S9', 'S7'}, ("{'S15', 'S6', 'S11', 'S10', 'S12', 'S13', 'S5', 'S9', 'S7'}", 'b'): {'S15', 'S6', 'S11', 'S10', 'S12', 'S13', 'S5', 'S9', 'S7'}, ("{'S15', 'S6', 'S11', 'S10', 'S12', 'S13', 'S5', 'S9', 'S7'}", 'd'): {'S16', 'S13', 'S15', 'S14'}, ("{'S8', 'S15', 'S11', 'S10', 'S12', 'S13', 'S5', 'S9', 'S7'}", 'c'): {'S8', 'S15', 'S11', 'S10', 'S12', 'S13', 'S5', '

In [173]:
from graphviz import Digraph
def visualize_dfa(D_transitions):
    dot = Digraph()

    # Add nodes
    for key in dict_of_states.keys():
        dot.node(str(new_names_states[str(key)]), shape='doublecircle' if dict_of_states[key] else 'circle')

    # Add edges
    for (src, symbol), dst in D_transitions.items():
        element = src.strip("{}")
        # Add quotes around the element and put it back into a string with curly braces
        #converted = "{'" + element + "'}"
        #src = "{'" + src.strip("{}").split(", ") + "'}"
        #dst = "{'" + dst.strip("{}").split(", ") + "'}"
        dot.edge(str(new_names_states[(src)]),str(new_names_states[str(dst)]), label=symbol)
    dot.edge('start' , 'S0')

    return dot


#D_transitions = convert_dfa_from_nfa(nfa , D_states , D_transitions)
dot = visualize_dfa(D_transitions)
dot.format = 'png'
dot.render('dfa_graph')

'dfa_graph.png'

In [15]:
# first_set = set()
# second_set = set()
# for key in dict_of_states.keys():
#     if dict_of_states[key]:
#         second_set.add(key)
#     else:
#         first_set.add(key)
# stack = [first_set, second_set]
# def DFA_Minimization(D_transitions, Symbols, dict_of_states):
#     P = [first_set, second_set]
#     W = [second_set]  
    
#     while W:
#         A = W.pop()
#         for c in Symbols:
#             X = set()
#             for state in dict_of_states.keys():
#                 if (state, c) in D_transitions:
#                     if str(D_transitions[(state, c)]) in A:
#                         X.add(state)
#             for Y in P[:]:
#                 intersection = X & Y
#                 difference = Y - X
#                 if intersection:
#                     P.remove(Y)
#                     P.extend([intersection, difference])
#                     if Y in W:
#                         W.remove(Y)
#                         W.extend([intersection, difference])
#                     else:
#                         if len(intersection) <= len(difference):
#                             W.append(intersection)
#                         else:
#                             W.append(difference)
#     for sets in P[:]:
#         if sets == set():
#             P.remove(sets)  
#     return P

# P = DFA_Minimization(D_transitions, Symbols, dict_of_states)
# li = list(dict_of_states.keys())
# node_mapper = {}
# for i in range(len(P)):
#     node_mapper[f'S{i}'] = (P[i])

# converted_dict = {}
# for key, value_set in node_mapper.items():
#     for value in value_set:
#         converted_dict[value] = key


# minimized_transitions = {}
# for (src, symbol), dst in D_transitions.items():
#     for i in range(len(P)):
#         if src in P[i]:
#             src = f'S{i}'
#             break
#     for i in range(len(P)):
#         if dst in P[i]:
#             dst = f'S{i}'
#             break
#     minimized_transitions[(src, symbol)] = converted_dict[str(dst)]
# print('Minimized Transitions: ',minimized_transitions)
# print("Node mapper : ",node_mapper)
# dict_of_min_states = {}
# for key,sets in node_mapper.items():
#     dict_of_min_states[key] = False
#     for set_ in sets:
#         if dict_of_states[set_]:
#             dict_of_min_states[key] = True
#             break
    
# print((dict_of_min_states))   
# print(P)
# print(len(P))    

Minimized Transitions:  {('{S1}', 'a'): 'S0', ('{S3, S2}', 'b'): 'S0', ('{S15, S11, S12, S5, S13, S9, S4, S7}', 'c'): 'S0', ('{S15, S11, S12, S5, S13, S9, S4, S7}', 'b'): 'S0', ('{S15, S11, S12, S5, S13, S9, S4, S7}', 'd'): 'S1', ('{S16, S13, S15, S14}', 'd'): 'S1', ('{S15, S6, S11, S10, S12, S13, S5, S9, S7}', 'c'): 'S0', ('{S15, S6, S11, S10, S12, S13, S5, S9, S7}', 'b'): 'S0', ('{S15, S6, S11, S10, S12, S13, S5, S9, S7}', 'd'): 'S1', ('{S8, S15, S11, S10, S12, S13, S5, S9, S7}', 'c'): 'S0', ('{S8, S15, S11, S10, S12, S13, S5, S9, S7}', 'b'): 'S0', ('{S8, S15, S11, S10, S12, S13, S5, S9, S7}', 'd'): 'S1'}
Node mapper :  {'S0': {"{'S3', 'S2'}", "{'S1'}", "{'S8', 'S15', 'S11', 'S10', 'S12', 'S13', 'S5', 'S9', 'S7'}", "{'S15', 'S6', 'S11', 'S10', 'S12', 'S13', 'S5', 'S9', 'S7'}", "{'S15', 'S11', 'S12', 'S5', 'S13', 'S9', 'S4', 'S7'}"}, 'S1': {"{'S16', 'S13', 'S15', 'S14'}"}}
{'S0': False, 'S1': True}
[{"{'S3', 'S2'}", "{'S1'}", "{'S8', 'S15', 'S11', 'S10', 'S12', 'S13', 'S5', 'S9', 'S7'

In [167]:
def minimize_dfa(D_transitions, dict_of_states):
    partitions = [set(), set()]
    for state, is_accepting in dict_of_states.items():
        partitions[is_accepting].add(state)     # Non accepting states then accepting states
    while True:
        new_partitions = []
        for partition in partitions:
            if len(partition) <= 1: 
                new_partitions.append(partition)
                continue

            transition_partitions = {}
            for state in partition:
                transitions = [(symbol, frozenset(D_transitions[state, symbol]) if ((state, symbol) in D_transitions.keys()) else None) for symbol in Symbols] 
                transition_partitions[state] = transitions
            grouped_transitions = {}
            for state, transitions in transition_partitions.items():
                if tuple(transitions) not in grouped_transitions.keys():
                    grouped_transitions[tuple(transitions)] = set()
                grouped_transitions[tuple(transitions)].add(state)

            new_partitions.extend(grouped_transitions.values())

        if new_partitions == partitions: 
            break
        partitions = new_partitions

    # Merge equivalent states
    state_mapping = {}
    for i, partition in enumerate(partitions):
        for state in partition:
            state_mapping[state] = f'Q{i}'

    return partitions , state_mapping

partitions , state_mapping = minimize_dfa(D_transitions, dict_of_states)
print("States : " , partitions)
print(len(partitions))
print("State_mapping: " , state_mapping)
node_mapper = {}
for key, value in state_mapping.items():
    if value not in node_mapper:
        node_mapper[value] = set()
    node_mapper[value].add(key)
print("Node mapper : " , node_mapper)
dict_of_min_states = {}
for key,sets in node_mapper.items():
    dict_of_min_states[key] = False
    for set_ in sets:
        if dict_of_states[set_]:
            dict_of_min_states[key] = True
            break
    
print((dict_of_min_states))




minimized_transitions = {}
for (src, symbol), dst in D_transitions.items():
    for i in range(len(partitions)):
        if src in partitions[i]:
            src = f'Q{i}'
            break
    for i in range(len(partitions)):
        if dst in partitions[i]:
            dst = f'Q{i}'
            break
    minimized_transitions[(src, symbol)] = state_mapping[str(dst)]
print('Minimized Transitions: ',minimized_transitions)
q_0 = epsilon_closure(nfa, start_state) 
start_index = 1000
br = False 
for state in partitions:
    if br:
        break
    for s in list(state):
        #print(str(s),q_0)
        if s == str(q_0):
            start_index = partitions.index(state)
            br = True
            break
start_state_minimized = f'Q{start_index}'
print(start_state_minimized)           

States :  [{"{'S3', 'S2'}"}, {"{'S1'}"}, {"{'S8', 'S15', 'S11', 'S10', 'S12', 'S13', 'S5', 'S9', 'S7'}", "{'S15', 'S11', 'S12', 'S5', 'S13', 'S9', 'S4', 'S7'}", "{'S15', 'S6', 'S11', 'S10', 'S12', 'S13', 'S5', 'S9', 'S7'}"}, {"{'S16', 'S13', 'S15', 'S14'}"}]
4
State_mapping:  {"{'S3', 'S2'}": 'Q0', "{'S1'}": 'Q1', "{'S8', 'S15', 'S11', 'S10', 'S12', 'S13', 'S5', 'S9', 'S7'}": 'Q2', "{'S15', 'S11', 'S12', 'S5', 'S13', 'S9', 'S4', 'S7'}": 'Q2', "{'S15', 'S6', 'S11', 'S10', 'S12', 'S13', 'S5', 'S9', 'S7'}": 'Q2', "{'S16', 'S13', 'S15', 'S14'}": 'Q3'}
Node mapper :  {'Q0': {"{'S3', 'S2'}"}, 'Q1': {"{'S1'}"}, 'Q2': {"{'S8', 'S15', 'S11', 'S10', 'S12', 'S13', 'S5', 'S9', 'S7'}", "{'S15', 'S11', 'S12', 'S5', 'S13', 'S9', 'S4', 'S7'}", "{'S15', 'S6', 'S11', 'S10', 'S12', 'S13', 'S5', 'S9', 'S7'}"}, 'Q3': {"{'S16', 'S13', 'S15', 'S14'}"}}
{'Q0': False, 'Q1': False, 'Q2': False, 'Q3': True}
Minimized Transitions:  {('Q1', 'a'): 'Q0', ('Q0', 'b'): 'Q2', ('Q2', 'c'): 'Q2', ('Q2', 'b'): 'Q2', ('Q2'

In [168]:
combined_transitions = {}
grouped_transitions = {}

# Group transitions by (source, destination) tuple
for transition, destination in minimized_transitions.items():
    if (transition[0], destination) not in grouped_transitions:
        grouped_transitions[(transition[0], destination)] = [transition[1]]
    else:
        grouped_transitions[(transition[0], destination)].append(transition[1])

# Combine symbols for each (source, destination) group
for (source, destination), symbols in grouped_transitions.items():
    combined_transitions[(source, ','.join(sorted(set(symbols))))] = destination

print("Combined Transitions:", combined_transitions)


Combined Transitions: {('Q1', 'a'): 'Q0', ('Q0', 'b'): 'Q2', ('Q2', 'b,c'): 'Q2', ('Q2', 'd'): 'Q3', ('Q3', 'd'): 'Q3'}


In [172]:
def visualize_min_dfa(D_transitions , dict_of_min_states):
    dot = Digraph()

    # Add nodes
    for key in dict_of_min_states.keys():
        dot.node(key, shape='doublecircle' if dict_of_min_states[key] else 'circle')
        
    # Add edges
    for (src, symbol), dst in D_transitions.items():
        dot.edge((src), (dst), label=symbol)
    dot.edge('start' , start_state_minimized)
    return dot

dot = visualize_min_dfa(combined_transitions , dict_of_min_states)
dot.format = 'png'
dot.render('dfa_min_graph')

'dfa_min_graph.png'